In [10]:
using Pkg
Pkg.add("Polynomials")
import Polynomials.Polynomial

   Resolving package versions...
  No Changes to `~/.local/share/julia/environments/v1.8/Project.toml`
  No Changes to `~/.local/share/julia/environments/v1.8/Manifest.toml`


In [12]:
Polynomial([1,2,3])

Polynomial(1 + 2*x + 3*x^2)